# GroupMe Analytics
## Data Mine your Group Chat

Prerequisites: 
* install groupy (pip install groupy)
* Be sure you're using Python3
* find your access token on [dev.groupme.com](https://dev.groupme.com/)  
* save it in your home directory as .groupy.key (echo 'your_access_token' > ~/.groupy.key )

In [ ]:
import groupy
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import sys, re
import numpy as np
from random import random
from operator import add
%matplotlib inline

In [ ]:
#note: need groupy api key saved in user's home directory for this to work
groups = groupy.Group.list()
print(groups)

In [ ]:
#replace the index below with whichever group you want to analyze from the above printed list
group = groups[1]
members = group.members()

## Preprocessing: feeding messages into a data frame + extracting features

### bonus points: all profanity printed below

In [ ]:
#populate messages list 
temp_msgs = []
messages = group.messages()
profanity_list = ['fuck', 'shit', 'fuk', 'damn', 'bitch', 'ass', ' ass ', ' ass.', 'douche', 'asshole', 'douchebag', 'fucker', 'dick', 'cunt', 'fucking']
profanity_list_2 = ['fuck', 'shit', 'fuk', 'damn', 'bitch', ' ass ', ' ass.', 'douche', 'asshole', 'douchebag', 'fucker', 'dick', 'cunt', 'fucking']

for i in range(1000):  #1000 is an arbitrary number. This can be decreased unless your group chat is huge.
    try:
        if len(messages) != 0:
            temp_msgs.append(messages)
            messages = messages.older()
    except:
        pass
    
msgs = []
for i in temp_msgs:
    for j in i:
        msgs.append(j)
msgs_2 = msgs
    
#put features into tuples 
for i in range(len(msgs)):
    ex = msgs[i]
    #print(ex.likes())
    self_like = 0
    fucks_given = 0
    profanity = 0
    selfname = ex.name
    for name in ex.likes():
        if str(name) == str(selfname):
            self_like = 1
    try:
        text = str(ex.text).lower()  #lowercase
    except:
        text = str(ex.text)
    if any(word in text for word in profanity_list_2):
        print(text)
    text2 = text.split()
    for word in text2:
        if(word in profanity_list):
            profanity = profanity + 1
    if 'fuck' in text:
        fucks_given = 1
    tup = (ex.name, ex.user_id, ex.created_at, text,ex.favorited_by, ex.likes(), len(ex.likes()), self_like, profanity, fucks_given, len(text2))
    msgs[i] = tup

### Create DataFrame from above processed data

In [ ]:
colnames = ['name', 'uid', 'datetime', 'text', 'favs', 'likes', 'num_likes', 'self_like', 'profanity', 'fucks_given', 'wordcount']
df = pd.DataFrame.from_records(msgs, columns = colnames)
#df.head()

## Most liked messages in the group chat

In [ ]:
print("most liked messages in groupme history")
df.sort_values(by="num_likes", ascending=False).head(10)

## Most profane messages in the group chat

In [ ]:
print("most profane messages")
df.sort_values(by="profanity", ascending=False).head(5)

## Create Users DataFrame for aggregate info

In [ ]:
users = pd.DataFrame(np.asarray(members), columns = ["name"])
#most active members

users['num_msgs'] = 0
users['mean_message_length'] = 0
users['num_likes'] = 0
users['self_likes'] = 0
users['profanity'] = 0
users['fucks_given'] = 0


for i in members:
    num_msgs = df[df['name']==str(i)].shape[0]
    users.loc[users['name'] == i, 'num_msgs'] = num_msgs
    wordcounts = df[df['name']==str(i)]['wordcount']
    wordcounts = np.mean(wordcounts)
    users.loc[users['name'] == i, 'mean_message_length'] = wordcounts
    num_likes = df[df['name']==str(i)]['num_likes'].sum()
    users.loc[users['name'] == i, 'num_likes'] = num_likes
    self_likes = df[df['name']==str(i)]['self_like'].sum()
    users.loc[users['name'] == i, 'self_likes'] = self_likes
    profanity = df[df['name']==str(i)]['profanity'].sum()
    users.loc[users['name'] == i, 'profanity'] = profanity
    fucks_given = df[df['name']==str(i)]['fucks_given'].sum()
    users.loc[users['name'] == i, 'fucks_given'] = fucks_given  
    
users['likes_per_msg'] = users['num_likes']/users['num_msgs']  

In [ ]:
users

## visualize features in the DataFrame

In [ ]:
#Bar chart of message counts
names = np.asarray(users['name'])
counts = np.asarray(users['num_msgs'])
for i in range(len(names)):
    names[i] = str(names[i])

plt.figure(figsize=(12,6))
plt.xticks(np.linspace(1,len(names), num=len(names)), names, rotation=30)
plt.bar(np.linspace(1,len(names), num=len(names)), counts)

## Analyzing Lexicon: Which words best define each member of the group chat? Which are the most commonly used?

In [ ]:
#TFIDF Scores 
#first, collapse everyone's messages into personalized documents 
documents = []
for i in members:
    document = ""
    msgs = df[df['name']==str(i)]['text']
    for i in msgs:
        document += (str(i) + " ")
    documents.append(document)


#create the vectorizer
tfidf = TfidfVectorizer(max_df=0.9,
                        ngram_range=(1, 1),
                        stop_words='english',
                        strip_accents='unicode', analyzer = 'word')

#run the multiplicaiton
tfidf_matrix =  tfidf.fit_transform(documents)
feature_names = tfidf.get_feature_names() 


## Most common words in the group

In [ ]:
indices = np.argsort(tfidf.idf_)#[::1]
tfidf_matrix_2 = tfidf_matrix.todense()
top_n = 100
top_features = [feature_names[i] for i in indices[:top_n]]
print("Most used words in the groupme: ")
print(top_features)

In [ ]:
i = 0
j = 0
dicts = []
for person in members:
    persondict = {}
    for word in feature_names:
        if tfidf_matrix[i,j] != 0:
            persondict[word] = tfidf_matrix[i,j]
        j += 1
    j = 0
    i += 1
    #print('person done')
    dicts.append(persondict)
#sorted(tfidf_matrix_2[5].tolist()[0])[::-1]

In [ ]:
import operator
j = 0
words_dict = {}
for i in dicts:
    sorted_vals = sorted(i.items(), key=operator.itemgetter(1), reverse=True)
    words_dict[str(members[j])] = sorted_vals[0:15]
    j += 1

## Most distinctive words per person 

* Determined by TF-IDF scores
* Essentially, the words that each person uses way more often than everyone else 

In [ ]:
for entry in words_dict:
    print("Top words for " + str(entry) + ": ")
    for w in words_dict[entry]:
        print(w)
    print('\n')

## Some time series stuff

### by month

In [ ]:
#Usage patterns
print("Groupme useage by month")
df.groupby(df.datetime.dt.month).count().plot(kind="bar", legend=False)

### by hour

In [ ]:
#Usage patterns
print("Groupme useage by hour of day")
df.groupby(df.datetime.dt.hour).count().plot(kind="bar", legend=False)

### running use frequency by day/hour since the group chat was created

In [ ]:
#note: this one takes a sec to complete if there is a large amount of data
print('Usage since inception')
n = 20

ax = df.groupby(df.datetime.dt.date).count().plot(kind="bar", legend=False, rot=30, fontsize=8, figsize=(14,6))
ticks = ax.xaxis.get_ticklocs()
ticklabels = [l.get_text() for l in ax.xaxis.get_ticklabels()]
ax.xaxis.set_ticks(ticks[::n])
ax.xaxis.set_ticklabels(ticklabels[::n])

ax.figure.show()